# 01. Data Validation

This notebook focuses on validating the raw data for Swedish electricity price forecasting. We will check for data integrity, time continuity, and basic statistical distributions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid", palette="muted")

# Add project root to sys.path for internal imports
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.utils import get_data_path, convert_to_swedish_time, load_data

# Define raw data directory
RAW_DATA_DIR = get_data_path('raw')
print(f"Raw data directory: {RAW_DATA_DIR}")

## 2. Check Data Integrity (Missing Values Statistics)
First, we need to understand if there are "holes" in the data, i.e., missing records.

- **Statistics**: Count the number and percentage of missing values.
- **Localization**: For time-series data, missing values often appear in specific time periods.

In [ ]:
def load_and_preprocess_entsoe(file_path):
    """Parses ENTSO-E formatted CSV files (Load data)."""
    df = pd.read_csv(file_path)
    
    # 1. Handle Time Column (MTU)
    # Format usually: "01/01/2024 01:00 - 01/01/2024 02:00"
    # Sometimes includes "(CET)" or "(CEST)" labels during DST transitions
    mtu_col = [c for c in df.columns if 'MTU' in c][0]
    df['timestamp_raw'] = df[mtu_col].str.split(' - ').str[0]
    
    # Remove DST labels like "(CET)" or "(CEST)" that cause parsing errors
    df['timestamp_clean'] = df['timestamp_raw'].str.replace(r'\(CE[S]?T\)', '', regex=True).str.strip()
    
    # Parse to naive datetime
    df['timestamp'] = pd.to_datetime(df['timestamp_clean'], dayfirst=True, format='mixed')
    
    # 2. Handle Area Column
    area_col = [c for c in df.columns if 'Area' in c][0]
    df['area'] = df[area_col].str.replace('BZN\\|', '', regex=True)
    
    # 3. Handle Value (Pick Load Forecast or Price)
    val_cols = [c for c in df.columns if 'Load' in c or 'Price' in c]
    val_col = val_cols[0]
    target_name = 'value'
    df = df.rename(columns={val_col: target_name})
    
    return df[['timestamp', 'area', target_name]]

# Load and combine CSV files
csv_files = list(RAW_DATA_DIR.glob("*.csv"))
if csv_files:
    df = pd.concat([load_and_preprocess_entsoe(f) for f in csv_files], ignore_index=True)
    
    # ENTSO-E data is localized (CET/CEST). 
    # To use our project utility correctly, we first localize it as Stockholm time.
    # This allows convert_to_swedish_time to handle it accurately without assuming UTC.
    df['timestamp'] = df.groupby('area')['timestamp'].transform(
        lambda x: x.dt.tz_localize('Europe/Stockholm', ambiguous='infer', nonexistent='shift_forward')
    )
else:
    print("No CSV files found. Loading Nord Pool price data from Parquet...")
    df = load_data('raw', 'nordpool_prices.parquet')

# Apply project-wide time conversion (Standardizes to Naive Stockholm time)
df = convert_to_swedish_time(df, 'timestamp')
df.sort_values(['area', 'timestamp'], inplace=True)
df.set_index('timestamp', inplace=True)

print("Missing Value Statistics:")
print(df.isnull().sum())

missing_pct = df.isnull().mean() * 100
print(f"\\nMissing Percentage (%):\\n{missing_pct}")

## 3. Check Time Continuity
Electricity data is typical time-series data. Any jumps or gaps in time will severely affect subsequent Lag Feature calculations.

- **Frequency Check**: Ensure the data matches the expected hourly ('H') frequency.
- **Breakpoint Detection**: Identify discontinuous points on the timeline.

In [ ]:
# Check for duplicate timestamps per area
duplicates = df.groupby('area').apply(lambda x: x.index.duplicated().sum())
print(f"Duplicate timestamp counts per area:\\n{duplicates}")

# Check for missing hours in the sequence
def check_missing_hours(group):
    # Remove duplicates for continuity check
    clean_index = group.index.drop_duplicates()
    full_range = pd.date_range(start=clean_index.min(), end=clean_index.max(), freq='H')
    missing = full_range.difference(clean_index)
    return len(missing), missing[:5]

continuity_results = df.groupby('area').apply(check_missing_hours)

print("\\nContinuity Analysis (Missing Hours):")
for area, (count, examples) in continuity_results.items():
    print(f"- Area {area}: {count} missing hours.")
    if count > 0:
        print(f"  Examples: {examples}")

## 4. Basic Descriptive Statistics
Identify outliers (negative prices or extreme peaks) and basic distributions.

- **Numerical Range**: Focus on min, max, mean, and quantiles.
- **Outlier Context**: Extreme values during weather events or energy crises need verification.

In [ ]:
print("Descriptive Statistics:")
print(df.describe())

# Boxplot for distribution visualization
plt.figure(figsize=(12, 7))
sns.boxplot(data=df.reset_index(), x='area', y='value', palette="magma")
plt.title(f"Value Distribution by Bidding Zone (SE1-SE4)", fontsize=14, fontweight='bold')
plt.xlabel("Bidding Zone", fontsize=12)
plt.ylabel("Value (Load/Price)", fontsize=12)
plt.tight_layout()
plt.show()

## 5. Verification Summary
Based on the analysis above, document the following:

- **Data Coverage**: Does it include all areas (SE1-SE4)? What is the full date range?
- **Data Quality**: Is the missing value ratio acceptable? Are there specific problematic gaps?
- **Next Steps**: Is interpolation required? Any obvious outliers to handle in the cleaning phase?